In [1]:
import os
import sys
import json

import numpy as np
import pandas as pd

import torch
import torchvision

import matplotlib.pyplot as plt
import lightning as L

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

from utils.model_utils import unwrap_model
from config.optimizer_param_config import optimizer_param_dict
from config.channel_param_config import channel_param_dict, enc_linear_param_dict

from data_module.data_module import ALDataModule_v1
from model.enc_classifier_model import AE1DClassifier, AE1DMaxPoolBNClassifierModel
from model.AE_model import AECNN1DBNModel, AE1DMaxPoolBNModel

/nfs/ksdata/tran/miniconda3/envs/python_3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import argparse

parser = argparse.ArgumentParser(description="", formatter_class=argparse.ArgumentDefaultsHelpFormatter)

parser.add_argument("-tl", "--train_limit_data", type=int, default=1000,
                    help=f"Used for limit the number of data in train data, -1 mean no limit")

parser.add_argument("-cn", "--class_num", type=int, default=8,
                    help=f"Number of class num use to train")

parser.add_argument("--model_name", type=str, default="4cnn_64-8-1000-64",
                    help=f"Config name of model used for training")

parser.add_argument("--AE_ckpt_path", type=str, default="/nfs/ksdata/tran/HAR_AE/lightning_logs/11.1_AE/4cnn_64-8-1000-64/version_0/checkpoints/sample_epoch=2269-step=9080-val_loss=0.000000.ckpt",
                    help=f"Config name of model used for training")

parser.add_argument("--linear_param", type=str, default="32",
                    help=f"Linear config name of model used for training")

parser.add_argument("-r", "--random_seed", type=int, default=42,
                    help=f"Random Seed")

parser.add_argument("--eval_mode", type=int, default=0,
                    help=f"Eval all models")

if os.path.basename(sys.argv[0]) == 'ipykernel_launcher.py':
    args = parser.parse_args(args=[])
else:
    args = parser.parse_args()

In [3]:
random_seed = 42
L.seed_everything(random_seed)

Seed set to 42


42

In [5]:
from lightning.pytorch.utilities.model_summary import ModelSummary

net = AE1DMaxPoolBNClassifierModel(
    AE1DMaxPoolBN_ckpt_path=args.AE_ckpt_path,
    linear_channel_param=enc_linear_param_dict[args.linear_param],
    out_features=8
).to("cpu")

model_summary = ModelSummary(net, max_depth=6)
print(model_summary)

   | Name                             | Type               | Params | In sizes     | Out sizes   
-------------------------------------------------------------------------------------------------------
0  | feature_extractor                | AE1DMaxPoolBNModel | 665 K  | ?            | ?           
1  | feature_extractor.enc_cnn        | Sequential         | 316 K  | [10, 6, 257] | [10, 256, 1]
2  | feature_extractor.enc_cnn.0      | Conv1d             | 1.6 K  | [10, 6, 257] | [10, 32, 84]
3  | feature_extractor.enc_cnn.1      | BatchNorm1d        | 64     | [10, 32, 84] | [10, 32, 84]
4  | feature_extractor.enc_cnn.2      | ReLU               | 0      | [10, 32, 84] | [10, 32, 84]
5  | feature_extractor.enc_cnn.3      | Conv1d             | 18.5 K | [10, 32, 84] | [10, 64, 26]
6  | feature_extractor.enc_cnn.4      | BatchNorm1d        | 128    | [10, 64, 26] | [10, 64, 26]
7  | feature_extractor.enc_cnn.5      | ReLU               | 0      | [10, 64, 26] | [10, 64, 26]
8  | feature_e

In [6]:
n_epochs = 20000
patience = n_epochs//100

batch_size = 512
optimizer, optimizer_param = optimizer_param_dict["Adam"]
dataset_path = f"dataset/processed_concat_data_{args.class_num}_labels"

log_save_dir = "lightning_logs"
log_save_name = f"15.1_enc_classify/{args.model_name}-{args.class_num}-{args.train_limit_data}-{args.random_seed}"

In [ ]:
data_module = ALDataModule_v1.load_from_checkpoint(args.AE_ckpt_path)
data_module.limit_and_set_train_data(data_module._train_data, data_module._train_label, limit_number=args.train_limit_data)

In [ ]:
L.seed_everything(args.random_seed)

net = AE1DMaxPoolBNClassifierModel(
    AE1DMaxPoolBN_ckpt_path=args.AE_ckpt_path,
    optimizer = optimizer,
    optimizer_param = optimizer_param, 
    linear_channel_param = enc_linear_param_dict[args.linear_param],
    out_features=8,
)

In [ ]:
print(" ----------------------start training---------------------------")
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger, CSVLogger
from lightning.pytorch.utilities.model_summary import ModelSummary

tensorboard_logger = TensorBoardLogger(save_dir=log_save_dir, name=log_save_name,)
csv_logger = CSVLogger(save_dir=log_save_dir, name=log_save_name,)
checkpoint_callback = ModelCheckpoint(
    dirpath=None,
    save_top_k=1,
    monitor="val_loss",
    mode="min",
    filename="sample_{epoch:02d}-{step:02d}-{val_loss:02f}"
)

trainer = L.Trainer(
    logger=[tensorboard_logger, csv_logger],
    callbacks=[EarlyStopping(monitor="val_loss", patience=patience), checkpoint_callback],
    max_epochs=n_epochs,
    check_val_every_n_epoch=10,
    accelerator="gpu", 
    devices=4, 
    strategy="ddp"
    )

trainer.fit(model=net, datamodule=data_module)
trainer_test_dict = trainer.logged_metrics

trainer.test(model=net, datamodule=data_module)
trainer_test_dict.update(trainer.logged_metrics)

for key in trainer_test_dict.keys():
    trainer_test_dict[key] = trainer_test_dict[key].item()

with open(os.path.join(trainer.logger.log_dir, "result.json"), "w") as f:
    json.dump(trainer_test_dict, f)

with open(os.path.join(trainer.logger.log_dir, "argparse_params.json"), "w") as f:
    json.dump(args.__dict__, f, indent=4)

In [ ]:
checkpoint_path = "/nfs/ksdata/tran/HAR_AE/lightning_logs/15.1_enc_classify/4cnn_64-8-1000-64-8-1000-42/version_0/checkpoints/sample_epoch=599-step=2400-val_loss=0.636736.ckpt"

data_module = ALDataModule_v1.load_from_checkpoint(checkpoint_path)
data_module.set_normal_train()
net = AE1DMaxPoolBNClassifierModel.load_from_checkpoint(checkpoint_path)

trainer = L.Trainer()

trainer.test(model=net, datamodule=data_module)
output = trainer.predict(model=net, datamodule=data_module)
pred = torch.argmax(torch.concat(output), dim=1)
gt = list(map(lambda x: x[1], iter(data_module.test_dataloader())))